In [1]:
import requests, sys
import pandas as pd

In [11]:
df = pd.read_csv(r"/home/vivak/human_demog_DFE/gff/GRCh37_functionalMasked.bed", sep='\t', names=['chrom','start','end','length'])
df = df[df.length>=500]
chroms = list(df.chrom)
starts = list(df.start)
ends = list(df.end)
#df.columns = ['#chr', 'start', 'end', 'length']
df.to_csv(r"/home/vivak/human_demog_DFE/gff/GRCh37_functionalMasked_500bp.bed", sep='\t', header=False, index=False)
#df = df.iloc[1305:]

In [4]:
fails = []
for i,j in enumerate(chroms):
    try:
        server = "https://rest.ensembl.org"
        ext = "/sequence/region/human/" + str(j) + ":" + str(starts[i]) + ".." + str(ends[i]) + ":1?coord_system_version=GRCh37"

        r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

        if not r.ok:
          r.raise_for_status()
          sys.exit()


        fasta = r.text
        with open(r"/home/vivak/human_demog_DFE/gff/fasta/"+ str(j) + "_" + str(starts[i]) + "_" + str(ends[i]) +".fasta", "w") as f:
            f.write(">" + str(j) + "\n")
            f.write(fasta)
    except Exception:
        fails.append([j, starts[i], ends[i]])

In [3]:
df = pd.read_csv(r"/home/vivak/human_demog_DFE/GRCh37_functionalMasked_15kb.bed", sep='\t', header=0)
chroms = list(df.chrom)
starts = list(df.start)
ends = list(df.end)

In [67]:
df = pd.read_csv(r"/home/vivak/human_demog_DFE/DFE_inference/GRCh37_genes.txt", sep='\t', header=0)
#df = df.drop_duplicates("name2")
df = df[['name2', 'chrom', 'txStart', 'txEnd']]
df.columns = ['gene', 'chrom', 'start', 'end']
df['gene_len'] = df.end - df.start

df2 = pd.read_csv(r"/home/vivak/human_demog_DFE/DFE_inference/human_chimp_orthologs.txt", sep='\t', header=0)
df2.columns = ['gene', 'pTro_gene']
lst = [x+1 for x in df2.index]
df2['region'] = lst
df = pd.merge(df2, df, on='gene', how='inner')

df3 = pd.read_csv(r"/home/vivak/human_demog_DFE/DFE_inference/GRCh37_exons_r_m.bed", sep='\t', header=0)
df3 = df3[['region','start', 'end']]
df3.columns = ['region', 'exon_start', 'exon_end']
df = pd.merge(df,df3,on='region',how='inner')

df['exon_start_in_gene'] = df.exon_start - df.start
df.head(10)

,gene,pTro_gene,region,chrom,start,end,gene_len,exon_start,exon_end,exon_start_in_gene
0,ENSG00000171621,ENSPTRG00000000114,1,chr1,9352938,9429591,76653,9427507,9429591,74569
1,ENSG00000171621,ENSPTRG00000000114,1,chr1,9354586,9416556,61970,9427507,9429591,72921
2,ENSG00000171621,ENSPTRG00000000114,1,chr1,9411489,9427922,16433,9427507,9429591,16018
3,ENSG00000171621,ENSPTRG00000000114,1,chr1,9415765,9427634,11869,9427507,9429591,11742
4,ENSG00000178585,ENSPTRG00000046205,2,chr1,9908333,9970321,61988,9908334,9910834,1
5,ENSG00000178585,ENSPTRG00000046205,2,chr1,9910166,9970394,60228,9908334,9910834,-1832
6,ENSG00000178585,ENSPTRG00000046205,2,chr1,9909596,9932150,22554,9908334,9910834,-1262
7,ENSG00000178585,ENSPTRG00000046205,2,chr1,9910507,9953295,42788,9908334,9910834,-2173
8,ENSG00000178585,ENSPTRG00000046205,2,chr1,9910716,9938234,27518,9908334,9910834,-2382
9,ENSG00000162458,ENSPTRG00000000208,3,chr1,16083101,16091568,8467,16111043,16113084,27942


In [83]:
from Bio import SeqIO
ids = []  
seq = []
s2 = []
for record in SeqIO.parse(r"/home/vivak/human_demog_DFE/DFE_inference/fasta/region"+ str(1) +"_exons.fasta", "fasta"):
    ids.append(record.id)
    seq.append(str(record.seq))
    s2.append(record.seq)
    
for record in SeqIO.parse(r"/home/vivak/human_demog_DFE/DFE_inference/fasta/ancestral/region"+ str(1) +"_exons.fasta", "fasta"):
    ids.append(record.id)
    seq.append(str(record.seq))
    s2.append(record.seq)

In [84]:
x = str(s2[0].translate())
y = str(s2[1].translate())
count = sum(1 for a, b in zip(x, y) if a != b)
count

TranslationError: Codon 'TT-' is invalid

In [89]:
str(s2[1])

'CCGAGCCgCTGCCGCTCATGGATTTGTGCCGTCGCTCGGTGCGCCTGGCCCTGGGGAAGGAGCGCCTGGGGGAGATCCACACGCTGCCGCTGCCGGCTTCCCTCAAGGCCTACCTCCTCTACCAGTGACGTTcGCCATCATACcGCCAGCGCGACAGCCACCTGGTGCCAACTCACTGAGCCGCCTGCCGCTGGGGCCGCCGCACCCTGCACCTTGGACCGGCATCCATAGCCATGGACAGAGGTCCCTGGtCTTCCCTCATCCTCCGTGGCTGCCTCCATGGGACAAGGACCgATTCCAACACAGGCTCCTCTTTCCCCCTTCCCGACATCAGCAGAAGGCAGCATCCCTGCATGCCGTCCATATACAACCCCTCTTTGAAAAAAGACACAGAGAATAAACTCCTACGAAAGCCCTACATTGAGCTCCAATCTGCTCGGGGTGGGACGGGTGCTTCCCACATCTCCGGGAGAAGGCTGCAGCCACCTGGGGGTCCCAGGGTGGTGGGGGTGGCAGGTGGTACCACAGCTCTGAGAGCAGATACCAGGGGTACTAAGAGGTGCTTAGACAAGGGCTGGTGCCCGGCCCAGGGTGCCCAGTGGGGCCATGCCATGGCAGATAAAGCTCAGGACGTCAAAAACTCACCATGGACACCAAGGCAGAAACCAAGAACTGTCTGCAGGCAAATAAGCACCCAGCACCCATCCtGGCTGCCGGTGCCCCGTACCCTATATTTATTCTTTTAACAATAACAAAAGCCATTTATTTATTCCATCTAGAAAGGAAACCCTGTTTCAGTCCCCTCTCTCTGGCTGTTCTGTTACTTTCCTTCCACCTGTGCCCTCCCTGGGATATGTATGCCTCGCCCGCCCTCCCTGGGCACATGTGCACACGTGCCCAGGCACAAGTATGTCTTTGGGTCCCTTGCCCTGCAGTTTCCAGGGGGCTCTGCTCCAAGTTCCCTAGCGGGCCCCTCAGGGAGAAATAGCCTCACGTGCA

In [79]:
re.findall('M[^M]*?_', a)

'GCACCCAGCACCCATCCTGGCTGCCGGTGCCCCGTACCCTGTATTTATTCTTTTAACAATAACAAAAGCCATTTATTTATTCCATCTAGAAAGGAAACCCTGTTTCAGTCCCCTCTCTCTGGCTGTTCTGT'

In [134]:
df = pd.read_csv(r"/home/vivak/human_demog_DFE/DFE_inference/GRCh37_exons_r_m.bed", sep='\t', header=0)
df['inter_end'] = df.end + (df['pi50']*4)
regions = list(df.region)
chroms = list(df.chrom)
starts = list(df.start)
ends = list(df.end)
inter_ends = list(df.inter_end)

In [135]:
df.m.describe()

count    3.970000e+02
mean     1.947462e-08
std      3.151216e-09
min      1.341703e-08
25%      1.740947e-08
50%      1.925783e-08
75%      2.117738e-08
max      4.075959e-08
Name: m, dtype: float64

In [12]:
fails = []
fails2 = []
fails3 = []
for i,j in enumerate(regions):
    #Attempt to get fasta of exon
    try:
        server = "https://rest.ensembl.org"
        ext = "/sequence/region/human/" + str(chroms[i]) + ":" + str(starts[i]) + ".." + str(ends[i]-1) + ":-1?coord_system_version=GRCh37"

        r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

        if not r.ok:
          r.raise_for_status()
          sys.exit()


        fasta = r.text
        with open(r"/home/vivak/human_demog_DFE/DFE_inference/fasta/region"+ str(j) +"_exon.fasta", "w") as f:
            f.write(">" + str(chroms[i]) + "\n")
            f.write(fasta)
    except Exception:
        fails.append([j, starts[i], ends[i]])
        
    #Attempt to get fasta of neighbouring intergenic region   
    try:
        server = "https://rest.ensembl.org"
        ext = "/sequence/region/human/" + str(chroms[i]) + ":" + str(ends[i]) + ".." + str(inter_ends[i]-1) + ":1?coord_system_version=GRCh37"

        r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

        if not r.ok:
          r.raise_for_status()
          sys.exit()


        fasta = r.text
        with open(r"/home/vivak/human_demog_DFE/DFE_inference/fasta/region"+ str(j) +"_intergenic.fasta", "w") as f:
            f.write(">" + str(chroms[i]) + "\n")
            f.write(fasta)
    except Exception:
        fails2.append([j, ends[i], inter_ends[i]])
        
    #Attempt to get fasta of exon + neighbouring intergenic region    
    try:
        server = "https://rest.ensembl.org"
        ext = "/sequence/region/human/" + str(chroms[i]) + ":" + str(starts[i]) + ".." + str(inter_ends[i]-1) + ":1?coord_system_version=GRCh37"

        r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

        if not r.ok:
          r.raise_for_status()
          sys.exit()


        fasta = r.text
        with open(r"/home/vivak/human_demog_DFE/DFE_inference/fasta/region"+ str(j) +"_exon_intergenic.fasta", "w") as f:
            f.write(">" + str(chroms[i]) + "\n")
            f.write(fasta)
    except Exception:
        fails3.append([j, starts[i], inter_ends[i]])

In [92]:
df = pd.read_csv(r"/home/vivak/human_demog_DFE/DFE_inference/GRCh37_exons_r_m.bed", sep='\t', header=0)
df['inter_end'] = df.end + (df['pi50']*4)
regions = list(df.region)
chroms = list(df.chrom)
starts = list(df.start)
ends = list(df.end)
inter_ends = list(df.inter_end)

In [101]:
hSap = []
for i,j in enumerate(regions):
    try:
        server = "https://rest.ensembl.org"
        ext = "/overlap/region/human/"+ str(chroms[i]) + ":" + str(starts[i]) + ".." + str(ends[i]) + "?coord_system_version=GRCh37;feature=gene;feature=exon"

        r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

        if not r.ok:
          r.raise_for_status()
          sys.exit()

        decoded = r.json()
        lst = repr(decoded).split(',')
        hSap.append([x for x in lst if "gene_id" in x][0].split(": ")[1][1:-1])
    except Exception:
        hSap.append('NA')

In [115]:
pTro = []
for i,j in enumerate(hSap):
    try:
        server = "https://rest.ensembl.org"
        ext = "/homology/id/human/"+j+"?target_species=chimpanzee;type=orthologues;sequence=none"

        r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

        if not r.ok:
          r.raise_for_status()
          sys.exit()

        decoded = r.json()
        lst = repr(decoded).split(',')
        pTro.append([x for x in lst if "ENSPTRG" in x][0].split(": ")[1][1:-1])
    except Exception:
        pTro.append('NA')

In [118]:
rdf = pd.DataFrame([hSap, pTro]).T
rdf.columns = ['hSap', 'pTro']
rdf.to_csv(r"/home/vivak/human_demog_DFE/DFE_inference/human_chimp_orthologs.txt", sep='\t', header=True, index=False)

In [151]:
df = pd.read_csv(r"/home/vivak/human_demog_DFE/DFE_inference/human_chimp_orthologs.txt", sep='\t', header=0)
df['region'] = df.index+1
df = df.dropna()
regions = list(df.region)
hSap = list(df.hSap)
pTro = list(df.pTro)

In [163]:
fails = []
for i,j in enumerate(regions):
    try:
        server = "https://rest.ensembl.org"
        ext = "/sequence/id/"+ hSap[i] + "?coord_system_version=GRCh37"

        r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

        if not r.ok:
          r.raise_for_status()
          sys.exit()

        hSap_seq = r.text

        server = "https://rest.ensembl.org"
        ext = "/sequence/id/"+ pTro[i] + "?coord_system_version=Pan_tro_3.0"

        r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

        if not r.ok:
          r.raise_for_status()
          sys.exit()

        pTro_seq = r.text

        with open(r"/home/vivak/human_demog_DFE/DFE_inference/human_chimp/region"+ str(j) +".fasta", "w") as f:
            f.write(">hSap\n")
            f.write(hSap_seq)
            f.write("\n")
            f.write(">pTro\n")
            f.write(pTro_seq)
    except Exception:
        fails3.append([j])

In [105]:
i=1
str(chroms[i]) + ":" + str(starts[i]) + ".." + str(ends[i])

'chr1:9908334..9910834'

In [117]:
i=0
server = "https://rest.ensembl.org"
ext = "/overlap/region/human/" + str(chroms[i]) + ":" + str(starts[i]) + ".." + str(ends[i]) + "?coord_system_version=GRCh37;feature=cds"
 
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
data = r.content
data_dict = json.loads(data)
decoded = r.json()
lst = repr(decoded)

In [124]:
for i in range(0, len(data_dict)):
    print(str(data_dict[i]['start']) + ' - ' + str(data_dict[i]['end']))

9415951 - 9416644
9427507 - 9427634
9415951 - 9416644
9427507 - 9427634
9415951 - 9416644
9427507 - 9427634


In [126]:
i=0
str(chroms[i]) + ":" + str(starts[i]) + ".." + str(ends[i])

'chr1:9427507..9429591'

In [130]:
server = "https://rest.ensembl.org"
ext = "/sequence/region/human/" + str(1) + ":" + str(9427504) + ".." + str(9427634) + ":1?coord_system_version=GRCh37"

r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

if not r.ok:
  r.raise_for_status()
  sys.exit()
fasta = r.text
print(fasta)

CAGCCGAGCCGCTGCCGCTCATGGATTTGTGCCGTCGCTCGGTGCGCCTGGCCCTGGGGAGGGAGCGCCTGGGGGAGATCCACACGCTGCCGCTGCCGGCTTCCCTCAAGGCCTACCTCCTCTACCAGTGA


In [132]:
server = "https://rest.ensembl.org"
ext = "/sequence/region/human/" + str(1) + ":" + str(9415951) + ".." + str(9416644) + ":1?coord_system_version=GRCh37"

r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

if not r.ok:
  r.raise_for_status()
  sys.exit()
fasta = r.text
print(fasta)

ATGGGTCAGAAGGTCACTGGAGGGATCAAGACTGTGGACATGAGGGACCCCACGTACAGGCCCCTGAAGCAGGAGCTCCAGGGTCTGGATTACTGCAAGCCCACCCGGCTGGATCTGCTACTGGACATGCCCCCTGTGTCCTATGATGTCCAGCTGCTGCATTCATGGAACAACAACGACCGATCGCTCAATGTCTTTGTGAAGGAGGACGACAAGCTCATCTTTCACCGGCATCCGGTGGCCCAGAGCACGGACGCTATCAGGGGCAAAGTCGGGTATACCCGTGGGCTGCACGTGTGGCAGATCACGTGGGCCATGAGACAGCGGGGCACACACGCCGTGGTGGGGGTGGCGACGGCAGACGCCCCCCTGCACTCTGTCGGGTACACAACCCTCGTGGGGAATAACCACGAGTCCTGGGGCTGGGACTTGGGGCGCAACCGGCTCTACCACGATGGCAAGAACCAGCCAAGCAAAACATACCCAGCCTTTCTGGAACCAGATGAGACATTCATTGTCCCTGACTCCTTCCTGGTAGCCCTGGACATGGACGACGGGACTCTGAGCTTCATTGTGGATGGACAGTACATGGGAGTGGCTTTTCGGGGACTCAAGGGCAAAAAACTGTATCCTGTAGTGAGTGCCGTCTGGGGCCACTGTGAGATCCGAATGCGCTACTTGAACGGACTCGATC


In [133]:
data_dict

[{'Parent': 'ENST00000328089',
  'start': 9415951,
  'strand': 1,
  'feature_type': 'cds',
  'seq_region_name': '1',
  'protein_id': 'ENSP00000330221',
  'phase': 0,
  'assembly_name': 'GRCh37',
  'source': 'ensembl_havana',
  'end': 9416644,
  'id': 'ENSP00000330221'},
 {'phase': '2',
  'assembly_name': 'GRCh37',
  'source': 'ensembl_havana',
  'end': 9427634,
  'id': 'ENSP00000330221',
  'Parent': 'ENST00000328089',
  'start': 9427507,
  'strand': 1,
  'protein_id': 'ENSP00000330221',
  'seq_region_name': '1',
  'feature_type': 'cds'},
 {'phase': 0,
  'source': 'havana',
  'assembly_name': 'GRCh37',
  'end': 9416644,
  'id': 'ENSP00000350573',
  'Parent': 'ENST00000357898',
  'start': 9415951,
  'strand': 1,
  'feature_type': 'cds',
  'seq_region_name': '1',
  'protein_id': 'ENSP00000350573'},
 {'feature_type': 'cds',
  'seq_region_name': '1',
  'protein_id': 'ENSP00000350573',
  'strand': 1,
  'start': 9427507,
  'Parent': 'ENST00000357898',
  'id': 'ENSP00000350573',
  'end': 94276